In [1]:
import keras
from keras import layers
import tensorflow as tf
import numpy as np
import os

In [2]:
# # training_dir = "../../../deeplearning/TensorFlow2.0_ResNet/dataset/train"
# training_dir = "/root/pythonprojects/deeplearning/covidDataset"
# datagen = tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.2, rescale=1. / 255.)
# height = 128
# width = 128
# channel = 1
# batch = 8
# train_it = datagen.flow_from_directory(training_dir, target_size = (height, width), color_mode="grayscale",
#                                       class_mode = "input", batch_size = batch, subset="training")
# val_it = datagen.flow_from_directory(training_dir, target_size = (height, width), color_mode="grayscale",
#                                       class_mode = "input", batch_size = batch, subset="validation")

In [3]:
%run read-image.ipynb
from glob import glob
path = "/root/pythonprojects/deeplearning/covidDataset"
Train_dir = os.path.join(path,"Train")
covid_image_path = glob(os.path.join(Train_dir,"COVID", "*.png"))
non_covid_image_path = glob(os.path.join(Train_dir,"non-COVID", "*.png"))
Test_dir = os.path.join(path,"Test")
covid_image_path_test = glob(os.path.join(Test_dir,"COVID", "*.png"))
non_covid_image_path_test = glob(os.path.join(Test_dir,"non-COVID", "*.png"))

train = []
test = []
size = 128
for path in covid_image_path:
    img = shape(path, flag1 = cv2.IMREAD_GRAYSCALE, flag2=True, size=size, plot=False)
    train.append(img)
for path in non_covid_image_path:
    img = shape(path, flag1 = cv2.IMREAD_GRAYSCALE, flag2=True, size=size, plot=False)
    train.append(img)
for path in covid_image_path_test:
    img = shape(path, flag1 = cv2.IMREAD_GRAYSCALE, flag2=True, size=size, plot=False)
    test.append(img)
for path in non_covid_image_path_test:
    img = shape(path, flag1 = cv2.IMREAD_GRAYSCALE, flag2=True, size=size, plot=False)
    test.append(img)

    
train = np.array(train)
test = np.array(test)

In [4]:
print(train.shape)
print(test.shape)

(2002, 128, 128)
(479, 128, 128)


In [5]:
x_train = train.astype('float32') / 255.
x_test = test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print(x_train.shape)
print(x_test.shape)

(2002, 16384)
(479, 16384)


In [6]:
# import matplotlib.pyplot as plt
# x, y = train_it.next()
# for i in range(0, 1):
#     image = x[i]
# #     print(y[i])
#     plt.imshow(image, cmap="gray")
#     plt.show()

In [7]:
input_img = keras.Input(shape=(16384,))
encoded = layers.Dense(128, activation='relu')(input_img)
encoded = layers.Dense(64, activation='relu')(encoded)
encoded = layers.Dense(32, activation='relu')(encoded)

decoded = layers.Dense(64, activation='relu')(encoded)
decoded = layers.Dense(128, activation='relu')(decoded)
decoded = layers.Dense(16384, activation='sigmoid')(decoded)

In [8]:
autoencoder = keras.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

autoencoder.fit(x_train, x_train,
                epochs=10,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test, x_test))

Epoch 1/10
16/16 [==============================] - 8s 120ms/step - loss: 0.6161 - val_loss: 0.5732
Epoch 2/10
16/16 [==============================] - 2s 96ms/step - loss: 0.5772 - val_loss: 0.5620
Epoch 3/10
16/16 [==============================] - 2s 97ms/step - loss: 0.5659 - val_loss: 0.5568
Epoch 4/10
16/16 [==============================] - 2s 97ms/step - loss: 0.5617 - val_loss: 0.5527
Epoch 5/10
16/16 [==============================] - 2s 117ms/step - loss: 0.5559 - val_loss: 0.5443
Epoch 6/10
16/16 [==============================] - 2s 108ms/step - loss: 0.5501 - val_loss: 0.5441
Epoch 7/10
16/16 [==============================] - 2s 98ms/step - loss: 0.5480 - val_loss: 0.5437
Epoch 8/10
16/16 [==============================] - 2s 99ms/step - loss: 0.5468 - val_loss: 0.5420
Epoch 9/10
16/16 [==============================] - 2s 98ms/step - loss: 0.5450 - val_loss: 0.5402
Epoch 10/10
16/16 [==============================] - 2s 98ms/step - loss: 0.5435 - val_loss: 0.5379


In [14]:
# autoencoder = keras.Model(input_img, decoded)
# encoder = keras.Model(input_img, encoded)
# encoding_dims = 32
# encoded_input = keras.Input(shape=(encoding_dims))
# decoder_layer = autoencoder.layers[-1]
# decoder = keras.Model(encoded_input, decoder_layer(encoded_input))

# encoded_imgs = encoded.predict(x_test)
# decoded_imgs = decoded.predict(encoded_imgs)
autoencoder.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 16384)]           0         
                                                                 
 dense (Dense)               (None, 128)               2097280   
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 64)                2112      
                                                                 
 dense_4 (Dense)             (None, 128)               8320      
                                                                 
 dense_5 (Dense)             (None, 16384)             2113

In [ ]:
import matplotlib.pyplot as plt

n = 10  # How many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    # Display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()